In [23]:
import numpy as np
import numpy.random as rn
import pandas as pd
import scipy.stats as st

In [24]:
df = pd.read_csv('final_data.csv')
df

,Unnamed: 0,group,logArea
0,1,immuno,-0.673345
1,2,immuno,2.197225
2,3,immuno,0.000000
3,4,immuno,-0.223144
4,5,immuno,-0.798508
...,...,...,...
114,115,cryo,0.000000
115,116,cryo,0.139762
116,117,cryo,-0.287682
117,118,cryo,-1.609438


In [3]:
np.unique(df['group'])

array(['cryo', 'immuno'], dtype=object)

In [4]:
X1 = df['logArea'][df['group']=='immuno']
X2 = df['logArea'][df['group']=='cryo']
X = [list(X1),list(X2)]

As
\begin{align}
\boldsymbol{\beta} \mid \gamma &\sim Dir(\gamma/L,\dots,\gamma/L)\\
\pi_j \mid \alpha_0,\boldsymbol{\beta} &\sim Dir(\alpha_0\boldsymbol{\beta})\\
z_{ji} \mid \boldsymbol{\pi}_j &\sim \boldsymbol{\pi}_j\\
\boldsymbol{\mu} &\sim H_{\mu}\\
\boldsymbol{\sigma^2} &\sim H_{\sigma^2}\\
x_{ji} \mid z_{ji},\boldsymbol{\mu},\boldsymbol{\sigma^2} &\sim N(\mu_{z_{ji}},\sigma_{z_{ji}})
\end{align}

### Complete Conditional distributions

Complete Conditional distribution of $z_{ij} \in \{1, \dots, L\}$
\begin{align}
P(z_{ji} \mid -) &\propto P(x_{ji} \mid z_{ji}, \boldsymbol{\mu}, \boldsymbol{\sigma}^2) \ P(z_{ji} \mid \boldsymbol{\pi}_j)\\
&\propto N(x_{ji};\mu_{z_{ji}},\sigma^2_{z_{ji}}) \ \pi_{jz_{ji}}\\
\Longrightarrow P(z_{ji} \mid -) &= \frac{N(x_{ji};\mu_{z_{ji}},\sigma^2_{z_{ji}}) \ \pi_{jz_{ji}}}{\sum^L_{k=1} N(x_{ji};\mu_k,\sigma^2_k) \ \pi_{jk}}
\end{align}

Complete Conditional distribution of $\pi_j$
\begin{align}
P(\pi_j \mid -) &\propto P(\pi_j \mid \alpha_0,\boldsymbol{\beta}) \ \prod^{n_j}_{i=1} P(z_{ji} \mid \pi_j)\\
&\propto \prod^L_{k=1} \pi_{jk}^{\alpha_0\beta_k-1} \ \ \prod^{n_j}_{i=1} \pi_{jz_{ji}}\\
&= \prod^L_{k=1} \pi_{jk}^{\alpha_0\beta_k-1+\sum^{n_j}_{i=1} \delta_k(z_{ji})}\\
&\sim Dir(\{\alpha_0\beta_k+\sum^{n_j}_{i=1} \delta_k(z_{ji})\}^L_{k=1})
\end{align}

Complete conditional distribution of $\boldsymbol{\beta}$
\begin{align}
P(\boldsymbol{\beta} \mid -) &\propto P(\boldsymbol\beta \mid \gamma) \ \prod^J_{j=1}P(\pi_j \mid \boldsymbol{\beta}, \alpha_0)\\
&\propto \prod^L_{k=1} \beta_k^{\gamma/L-1} \ \prod^J_{j=1}\prod^L_{k=1} \pi_{jk}^{\alpha_0\beta_k-1}\\
&=\prod^L_{k=1}( \beta_k^{\gamma/L-1}  (\prod^J_{j=1} \pi_{jk})^{\alpha_0\beta_k-1})\\
\end{align}

Complete conditional distribution of $\mu_k$
\begin{align}
P(\mu_k \mid -) &\propto P(\mu_k \mid H_\mu) \ \prod^J_{j=1}\prod^{n_j}_{i=1} P(x_{ji} \mid z_{ji},\mu_{1:L},\sigma^2_{1:L})\\
&\propto H_{\mu}(\mu_k) \prod^J_{j=1}\prod^{n_j}_{i=1} N(x_{ji};\mu_{z_{ji}},\sigma^2_{z_{ji}})
\end{align}

Complete conditional distribution of $\sigma^2_k$
\begin{align}
P(\sigma^2_k \mid -) &\propto P(\sigma^2_k \mid H_{\sigma^2}) \ \prod^J_{j=1}\prod^{n_j}_{i=1} P(x_{ji} \mid z_{ji},\mu_{1:L},\sigma^2_{1:L})\\
&\propto H_{\sigma^2}(\sigma^2_k) \prod^J_{j=1}\prod^{n_j}_{i=1} N(x_{ji};\mu_{z_{ji}},\sigma^2_{z_{ji}})
\end{align}

In [6]:
class HDPModel:
    
    def __init__(self, X, H_mu, H_sigma2, L, alpha_0 = 1, gamma = 1):
        '''
        Parameters:
        
        X (two dimentional List): Observations (X_ji stands for i-th data point in j-th groups)
        
        H_mu (function): pdf function of prior mu
        
        H_sigma2 (function): pdf function of prior sigma^2        
        
        L (positive integer): Number of mixture component
        
        alpha_0, gamma: concentration parameters
        '''
        
        self.X = X
        self.L = L
        self.H_mu = H_mu
        self.H_sigma2 = H_sigma2
        self.alpha_0 = alpha_0
        self.gamma = gamma
        
        self.groupNumber = len(X)   #Number of groups
        self.obsInGroup = [len(X[i]) for i in range(len(X))]  #Number of observations in each group
        
        #Initialize parameters
        self.beta = np.repeat(1/L,L)
        self.pi = [np.repeat(1/L,L) for gn in range(len(X))]
        self.Z = [rn.choice(range(L),self.obsInGroup[i]) for i in range(len(X))]   #Same structure as X
        self.mu = np.zeros(L)
        self.sigma2 = np.ones(L)
    
    
    #Update Z_ji via complete conditional distribution
    def update_Z(self,j,i): 
        prob = np.exp(st.norm.logpdf(self.X[j][i],self.mu,np.sqrt(self.sigma2))+np.log(self.pi[j]))
        prob /= sum(prob)  
        self.Z[j][i] = rn.choice(range(self.L),p=prob)
        return
    
    
    #Update pi_j via Complete conditional distribution
    def update_pi(self,j):
        count = np.zeros(self.L)
        for i in range(self.obsInGroup[j]):
            count[self.Z[j][i]] +=1
        self.pi[j] = st.dirichlet.rvs(self.alpha_0*self.beta+count)
        return
    
    
    #Complete conditional pdf of beta WITHOUT normalization
    def P_beta(self,beta):
        return np.exp(sum((self.gamma/self.L-1)*np.log(beta)
                          +(self.alpha_0*beta-1)*sum(np.log(self.pi))))
    
    #Update beta
    def update_beta(self):
        
        '''
        Slice Samplier
        '''
        return
    
    
    #Complete conditional pdf of mu_k WITHOUT normalization
    def P_mu(self,mu_k,k):
        lognorm=0
        for j in range(self.groupNumber):
            for i in range(self.obsInGroup[j]):
                if self.Z[j][i] != k:
                    lognorm += st.norm.logpdf(self.X[j][i], mu_k, np.sqrt(self.sigma2[self.Z[j][i]]))
                else:
                    lognorm += st.norm.logpdf(self.X[j][i], self.mu[self.Z[j][i]], np.sqrt(self.sigma2[self.Z[j][i]]))
        return np.exp(np.log(self.H_mu(mu_k)) + lognorm)
    
    
    #Update mu_k
    def update_mu(self,k):
        
        '''
        Slice Samplier
        '''
        
        return
    
    
    #Complete conditional pdf of sigma_k WITHOUT normalization
    def P_sigma2(self,sigma2_k,k):
        lognorm=0
        for j in range(self.groupNumber):
            for i in range(self.obsInGroup[j]):
                if self.Z[j][i] != k:
                    lognorm += st.norm.logpdf(self.X[j][i], self.mu[self.Z[j][i]], sigma2_k)
                else:
                    lognorm += st.norm.logpdf(self.X[j][i], self.mu[self.Z[j][i]], np.sqrt(self.sigma2[self.Z[j][i]]))
        return np.exp(np.log(self.H_sigma2(sigma2_k)) + lognorm)
    
    
    #Update sigma_k
    def update_sigma2(self,k):
        
        '''
        Slice Samplier
        '''
        
        return
    
    

    
    def train(self):
        
        '''
        Training the model through Gibbs Samplier, 
        with the full conditional distribution above
        '''
        
        return

In [ ]:
model = HDPModel(...)

Let the parameter $\phi_k = (\mu_k,\sigma_k^2)$

The likelihood
$$X_{ji} \mid \boldsymbol{\phi} \sim N(\mu_{k_{ji}},\sigma^2_{k_{ji}})$$

The priors $\phi_k \sim H$ such that $\mu_k \perp \!\!\! \perp \sigma^2_k$, with
$$\mu_k \mid \sigma^2_k \sim H_\mu = N(0,\sigma^2_k)\\
\sigma^2_k \sim H_{\sigma^2} = InvGamma(1,1)$$ 

Then the conditional density
\begin{align}
f^{-x_{ji}}_k(x_{ji}) 
&\propto \int \int \ h_\mu(\mu_k) \ h_{\sigma^2}(\sigma^2_k) f(x_{ji} \mid \mu_k, \sigma^2_k) \prod_{(j',i') \neq (j,i), k_{j't_{j'i'}} =k} f(x_{j'i'} \mid \mu_k, \sigma^2_k) \ d\mu_k d\sigma^2_k\\
&\propto  \int \int e^{-\frac{\mu_k^2}{2\sigma^2_k}} \ (\sigma^2_k)^{-2} e^{-\frac{1}{\sigma^2_k}} \ (\sigma_k^2)^{-\frac{1}{2}} \ e^{-\frac{(x_{ji}-\mu_k)^2}{2\sigma^2_k}} \prod_{(j',i') \neq (j,i),k_{j't_{j'i'}} =k} (\sigma_{k}^2)^{-\frac{1}{2}} \ e^{-\frac{(x_{j'i'}-\mu_{k})^2}{2\sigma^2_{k}}} \  \ d\mu_k d\sigma^2_k\\
&= \int \int (\sigma^2_k)^{-\frac{5}{2}-\frac{1}{2}\sum_{(j',i') \neq (j,i)}\delta_k(k_{j't_{j'i'}})} e^{-\frac{\mu_k^2 (1+\sum_{j',i'}\delta_k(k_{j't_{j'i'}})) \ -2\mu_k \sum_{j',i'}x_{j'i'}\delta_k(k_{j't_{j'i'}}) \  +\sum_{j',i'}x^2_{j'i'}\delta_k(k_{j't_{j'i'}}) \ +2}{2\sigma^2_k}} d\mu_k d\sigma^2_k\\
&= \int \int (\sigma^2_k)^{-\frac{5}{2}-\frac{1}{2}\sum_{(j',i') \neq (j,i)}\delta_k(k_{j't_{j'i'}})} e^{-\frac{\mu_k^2 (1+\sum_{j',i'}\delta_k(k_{j't_{j'i'}})) \ -2\mu_k \sum_{j',i'}x_{j'i'}\delta_k(k_{j't_{j'i'}}) \  +\sum_{j',i'}x^2_{j'i'}\delta_k(k_{j't_{j'i'}}) \ +2}{2\sigma^2_k}} d\mu_k d\sigma^2_k\\
\end{align}

Let $y_k = \sum_{j',i'}\delta_k(k_{j't_{j'i'}})$, then $\sum_{(j',i') \neq (j,i)} \delta_k(k_{j't_{j'i'}}) = y_k -1
$.

Let $y_{kx} = \sum_{j',i'}x_{j'i'}\delta_k(k_{j't_{j'i'}})$, and $y_{kx2} = \sum_{j',i'}x^2_{j'i'}\delta_k(k_{j't_{j'i'}})$

Then we have
\begin{align}
f^{-x_{ji}}_k(x_{ji}) 
&\propto \int \int (\sigma^2_k)^{-2-\frac{y_k}{2}} e^{-\frac{\mu_k^2 (1+y_k) \ -2\mu_k y_{kx} \  +y_{kx2} \ +2}{2\sigma^2_k}} d\mu_k d\sigma^2_k\\
&=\int \int (\sigma^2_k)^{-2-\frac{y_k}{2}} e^{-\frac{ (1+y_k)(\mu_k-\frac{y_{kx}}{1+y_k})^2 - \ \frac{y^2_{kx}}{1+y_k}  +y_{kx2} \ +2}{2\sigma^2_k}} d\mu_k d\sigma^2_k\\
&= \int (\sigma^2_k)^{-2-\frac{y_k}{2}} e^{-\frac{- \ \frac{y^2_{kx}}{1+y_k}  +y_{kx2} \ +2}{2\sigma^2_k}} \int e^{-\frac{(\mu_k-\frac{y_{kx}}{1+y_k})^2}{\frac{2\sigma^2_k}{1+y_k}}} d\mu_k d\sigma^2_k\\
&\propto \int (\sigma^2_k)^{-2-\frac{y_k}{2}} e^{-\frac{- \ \frac{y^2_{kx}}{1+y_k}  +y_{kx2} \ +2}{2\sigma^2_k}} \sqrt{\frac{\sigma^2_k}{1+y_k}} \int N(\mu_k;\frac{y_{kx}}{1+y_k},\frac{\sigma^2_k}{1+y_k}) d\mu_k d\sigma^2_k\\
&=\frac{1}{\sqrt{1+y_k}}\int (\sigma^2_k)^{-\frac{3}{2}-\frac{y_k}{2}} e^{-\frac{- \ \frac{y^2_{kx}}{1+y_k}  +y_{kx2} \ +2}{2\sigma^2_k}}  d\sigma^2_k\\
&= \frac{\Gamma(\frac{1}{2}+\frac{y_k}{2})}{\sqrt{1+y_k}(\frac{\ \frac{y^2_{kx}}{1+y_k}  +y_{kx2} \ +2}{2})^{\frac{1}{2}+\frac{y_k}{2}}}\\
&\propto \frac{1}{(\frac{y^2_{kx}}{1+y_k}  +y_{kx2} \ +2)^{\frac{1}{2}+\frac{y_k}{2}}}
\end{align}

$', \prime$

## Chinese Restaurant Franchise

In [ ]:
def CRF(nSample, X, K, H_mu, H_sigma2, alpha_0 = 1, gamma = 1):
    '''
    Args:
    
      nSample (int): Number of samples of theta of CRF
      
      X: data
      
      K (int): Number of global menus
      
      H_mu (function): draw one sample from prior of mu
      
      H_sigma2 (function): draw one sample from prior of sigma2
      
      alpha_0, gamma: concentration parameters
    '''
    tkSamples = []
    J = len(X)   #Number of groups
    n_j = [len(X[i]) for i in range(len(X))]  #Number of observations in each group
    
    #Initialize phi
    phi = []
    for k in range(K):
        phi.append((H_mu(),H_sigma2()))
        
    n_jt = [[] for j in range(J)]                      #number of customers sits at table t in restaurant j
    
    m_jk = [[[0] for k in range(K)] for j in range(J)] #number of tables in restaurant j serving dish k
    m_j = [[0] for j in range(J)]                      #number of tables in restaurant j
    m_k = [[0] for k in range(K)]                      #number of tables serving dish k
    m = 0                                              #total number of table
    
    for ns in range(nSample):
        sample = {'t_ji':[[] for j in range(J)],  #table index where customer i in restaurant j sits
                  'k_jt':[[] for j in range(J)]}  #dish where table t in restaurant j serves
        
        #Sample t
        for j in range(J):
            for i in range(n_j[j]):
                #draw a sample from Eq.24
                prob = np.array(n_jt[j] + [alpha_0]) 
                prob /= sum(prob) #Normalize
                t = rn.choice(range(len(prob)), p=prob)
                
                if t != len(prob) - 1: #Sample from an existing table
                    theta[j].append(psi_jt[j][t])
                    
                    
        
    
    
    
        
    
    
    